In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'

# Some features will be excluded and values of Yield lower than 0.1 will be dropped compared to model 2

# Exploration and Cleaning

In [3]:
def cleaning():
    df = pd.read_csv('crop_yield.csv', header=0)
    df.drop(columns=['Crop_Year', 'Production'], inplace=True)
    df = df[(df['Yield'] > 0.1) & (df['Yield'] <= 50)]
    df = df[df["Crop"] != 'Coconut']
    return df


,ID,Crop_Year,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
ID,1.000000,0.390021,0.001170,-0.067046,-0.142394,0.024655,0.016993,-0.068810
Crop_Year,0.390021,1.000000,-0.035686,0.003366,-0.011187,0.011169,-0.004657,0.002539
Area,0.001170,-0.035686,1.000000,0.037441,-0.106054,0.973255,0.973479,0.001858
Production,-0.067046,0.003366,0.037441,1.000000,0.029879,0.039799,0.035171,0.570809
Annual_Rainfall,-0.142394,-0.011187,-0.106054,0.029879,1.000000,-0.109734,-0.097657,0.020761
Fertilizer,0.024655,0.011169,0.973255,0.039799,-0.109734,1.000000,0.954991,0.002862
Pesticide,0.016993,-0.004657,0.973479,0.035171,-0.097657,0.954991,1.000000,0.001782
Yield,-0.068810,0.002539,0.001858,0.570809,0.020761,0.002862,0.001782,1.000000


# Split Test_Train

In [ ]:
from sklearn.model_selection import train_test_split

df = cleaning()

print(df.shape)

# Define the feature matrix and target variable
X = df.drop('Yield', axis=1)
y = df['Yield']

# Stratified split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=X['Crop'], random_state=42)


X_test


mdoel = Linear 

model.fit(X_trai, Y_train)

model.predic(X_test)

# Processing X_train

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.utils.validation import check_is_fitted
import numpy as np
import pandas as pd

# Custom transformer for frequency encoding
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.freq_dict_ = {}

    def fit(self, X, y=None):
        # Compute the frequency of each category in each column
        X_df = pd.DataFrame(X)  # Convert numpy array to DataFrame
        self.freq_dict_ = {col: X_df[col].value_counts(normalize=True).to_dict() for col in X_df.columns}
        return self

    def transform(self, X, y=None):
        check_is_fitted(self)
        X_transformed = pd.DataFrame(X).copy()  # Convert numpy array to DataFrame
        # Replace each category with its frequency
        for col in X_transformed.columns:
            X_transformed[col] = X_transformed[col].map(self.freq_dict_[col])
        return X_transformed.values  # Return numpy array

# Pipelines for frequency encoding and one hot encoding
freq_enc_pipeline = Pipeline(steps=[('freq_enc', FrequencyEncoder())])
one_hot_enc_pipeline = Pipeline(steps=[('one_hot_enc', OneHotEncoder(sparse_output=False))])

# Function to compute ratio, perform logarithmic transformation, and apply Standard Scaling
def compute_ratio_log_drop(df):
    df_copy = df.copy()
    df_copy["Log_Pesticide_per_hectar"] = np.log1p(df["Pesticide"] / df["Area"])
    df_copy["Log_Fertilizer_per_hectar"] = np.log1p(df["Fertilizer"] / df["Area"])
    df_copy = df_copy.drop(columns=["Pesticide", "Fertilizer", "Area"])
    return df_copy

# Function to perform logarithmic transformation and apply Standard Scaling
def log_drop(df):
    df_copy = df.copy()
    df_copy["Log_annual_rainfall"] = np.log1p(df["Annual_Rainfall"])
    df_copy = df_copy.drop(columns=["Annual_Rainfall"])
    return df_copy

ratio_pipeline = make_pipeline(FunctionTransformer(compute_ratio_log_drop), StandardScaler())

log_pipiline = make_pipeline(FunctionTransformer(log_drop), StandardScaler())

# Update the preprocessing pipeline
preprocessing = ColumnTransformer([
    ("compute_ratio_log_and_scale", ratio_pipeline, ["Pesticide", "Area", "Fertilizer"]),
    ("log_transform_and_scale", log_pipiline, ["Annual_Rainfall"]),
    ("freq_enc", freq_enc_pipeline, ["Crop", "State"]),
    ("one_hot_enc", one_hot_enc_pipeline, ["Season"])
])

# Fit the transformer to the training data
preprocessing.fit(X_train)

# Transform the training data
X_train_transformed = preprocessing.transform(X_train)

# Get the feature names from the transformers
transformed_feature_names = (
    ["Log_Pesticide_per_hectar_scaled", "Log_Fertilizer_per_hectar_scaled", "Log_annual_rainfall_scaled"]  # Ratio and log columns
    + ["freq_enc_Crop", "freq_enc_State"]  # Frequency encoded columns
    + list(preprocessing.named_transformers_["one_hot_enc"].named_steps["one_hot_enc"].get_feature_names_out(input_features=["Season"]))  # One hot encoded columns
)

# Create a DataFrame from the transformed training data
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=transformed_feature_names)


# RandSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

full_pipeline = Pipeline([("preprocessing", preprocessing), ("random_forest", RandomForestRegressor(random_state=42))])


param_dist = {
    'random_forest__n_estimators': randint(low=100, high=400),
    'random_forest__max_features': [1.0 , 'sqrt', 'log2'],
    'random_forest__max_depth': [10, 50, 100, None],
    'random_forest__min_samples_split': randint(low=2, high=10),
    'random_forest__min_samples_leaf': randint(low=1, high=3),
    'random_forest__bootstrap': [True, False]
}

rnd_search = RandomizedSearchCV(full_pipeline, param_distributions=param_dist, n_iter=60, cv=4, scoring='neg_root_mean_squared_error', random_state=1, n_jobs=-1, verbose=2)

y_train_log = np.log1p(y_train)

rnd_search.fit(X_train, y_train_log)

final_model_3 = rnd_search.best_estimator_

In [ ]:
feature_importance = final_model_3["random_forest"].feature_importances_

feature_importance.round(2)

rnd_search.best_params_, feature_importance

In [ ]:
from sklearn.model_selection import cross_val_score

Rmse = -cross_val_score(final_model_3, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)

pd.Series(Rmse).describe()

# Testing the model

In [ ]:
from sklearn.metrics import mean_squared_error

final_pred_log = final_model_3.predict(X_test)

final_pred = np.expm1(final_pred_log)

final_rmse = mean_squared_error(y_test, final_pred, squared=False)

final_rmse
# Result : 2.069048910404624



In [ ]:


# Convert numpy arrays to pandas Series
final_pred_series = pd.Series(final_pred, name="Predicted", index=y_test.index)
y_test_series = pd.Series(y_test, name="Actual")

# Create a new figure for the scatter plot of the predictions
plt.figure(figsize=(10, 10))
plt.scatter(y_test_series, final_pred_series, alpha=0.5)
plt.plot([y_test_series.min(), y_test_series.max()], [y_test_series.min(), y_test_series.max()], 'k--', lw=2, label='Perfect Prediction')
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs Actual Values")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import joblib

joblib.dump(final_model_3, "Model_vesrion_3_Log.pkl")

In [ ]:
df = pd.DataFrame({"Actual_3": y_test, "Predicted_3": final_pred})

df[["Resid_3", "Resid_Norm_3"]] = pd.DataFrame({"Resid_3": df["Actual_3"] - df["Predicted_3"], "Resid_Norm_3": (df["Predicted_3"]/df["Actual_3"])-1})


df.to_csv("Predictions_Model_3.csv")



In [ ]:
predictions = final_model_3.predict(X)


check = pd.DataFrame({"Actual_3": y,"Predicted_3": predictions})

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from math import sqrt


def calculate_metrics_as_df(df, model_name):
    actual_col = f'Actual_{model_name}'
    predicted_col = f'Predicted_{model_name}'
    actual = df[actual_col]
    predicted = df[predicted_col]
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = sqrt(mse)
    r2 = r2_score(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted)
    return pd.DataFrame({'Model': f'Model_{model_name}', 'MAE': [mae], 'MSE': [mse], 'RMSE': [rmse], 'R2': [r2], 'MAPE': [mape]})


metrics = calculate_metrics_as_df(check, 3)

metrics
